# WHOOP Data Ingest

Ensure we can reliably:
- Authorize using WHOOP (and figure out what information we need from users)
- Collect the proper metricts
- Format the metrics correctly for our prompt

This notebooks epxlores how we can handle the above, and will condense all the methods/code here into a single WHOOP Resource for reusability.

In [1]:
import sys 
import subprocess
import logging 

# get root of current repo and add to our path
root_dir = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], stderr=subprocess.DEVNULL).decode("utf-8").strip()

sys.path.append(root_dir)

# logging config
logging.basicConfig(
    level=logging.INFO,  # Show DEBUG and above
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)

logger = logging.getLogger(__name__)

## Authorization 

The WHOOP API uses OAuth 2.0 for user-client authentication. 

I'm still working out how OAuth works, but from where I stand now, it seems like we'd want to:
- Use our Client ID and Client Secret to request an access token which allows us to view data. 
- Additionally, use that ID, Secret, and Access Token to request a refresh token. This will? enable use to have users authenticate at the redirect once, then effectively "save" that response and use the refresh token to gain subsequent access.

In [14]:
import json 

def load_config(filepath: str = "config.json") -> dict:
    try:
        with open(filepath, "r") as f:
            config = json.load(f)
        logger.info(f"Configuration loaded from {filepath}")
        return config
    except FileNotFoundError:
        logger.error(f"Configuration file {filepath} not found.")
        return {}
    except json.JSONDecodeError as e:
        logger.error(f"Error decoding JSON from {filepath}: {e}")
        return {}
    except Exception as e:
        logger.error(f"Unexpected error loading config: {e}")
        return {}

In [39]:
import socketserver
import secrets
from urllib.parse import urlencode
import webbrowser

from utils.auth import OAuthCallbackHandler

# fetch a callback code
with socketserver.TCPServer(("", 8080), OAuthCallbackHandler) as httpd:
    # load our base config file 
    params = load_config(f"{root_dir}/config.json")

    # remove client secret from config, we dont need it
    params.pop("client_secret")

    # add scope, state, and response type to our params 
    params["scope"] = " ".join(["read:recovery", "read:cycles", "read:workout", "read:sleep", "read:profile", "offline"])
    params["state"] = secrets.token_urlsafe(16)
    params["response_type"] = "code"

    # build auth url 
    url = f"https://api.prod.whoop.com/oauth/oauth2/auth?{urlencode(params)}"

    # redirect for auth 
    webbrowser.open(url)
    httpd.handle_request()

    callback_code = OAuthCallbackHandler.callback_code

2025-09-19 11:43:05,659 [INFO] __main__: Configuration loaded from /home/srmarshall/code/meathead-mode/config.json


127.0.0.1 - - [19/Sep/2025 11:43:11] "GET /callback?code=NCM1zJnotzaW4Jgp5SuRZlgXxiB5Tfo8eI9sAf7xqFI.nE4BJDw_g4x6q7IxpFYdJdUoIYTqrcgti6ag2L70dWk&scope=read%3Arecovery%20read%3Acycles%20read%3Aworkout%20read%3Asleep%20read%3Aprofile%20offline&state=v_APZIvbIAyuTc5qz1xwdw HTTP/1.1" 200 -


In [40]:
import requests
import json 

# load our config items from our json file 
params = load_config(f"{root_dir}/config.json")

# add some additional params to the base config 
params["grant_type"] = "authorization_code"
params["code"] = callback_code

res = requests.post(
    url="https://api.prod.whoop.com/oauth/oauth2/token",
    data=params
)

# update config file
params.pop("grant_type")
params.pop("code")

params["access_token"] = res.json().get("access_token")
params["refresh_token"] = res.json().get("refresh_token")

with open(f"{root_dir}/config.json", "w") as f:
    json.dump(params, f, indent=4)

2025-09-19 11:43:13,233 [INFO] __main__: Configuration loaded from /home/srmarshall/code/meathead-mode/config.json
2025-09-19 11:43:13,234 [DEBUG] urllib3.connectionpool: Starting new HTTPS connection (1): api.prod.whoop.com:443
2025-09-19 11:43:13,612 [DEBUG] urllib3.connectionpool: https://api.prod.whoop.com:443 "POST /oauth/oauth2/token HTTP/1.1" 200 None


In [ ]:
def refresh_tokens(config_filepath: str = "config.json") -> None:

    params = load_config(config_filepath)

    # set request params
    refresh_params = {
        "grant_type": "refresh_token", 
        "client_id": params.get("client_id"),
        "client_secret": params.get("client_secret"),
        "scope": "offline", 
        "refresh_token": params.get("refresh_token")
    }

    # request new tokens
    res = requests.post(
        url="https://api.prod.whoop.com/oauth/oauth2/token",
        data=refresh_params
    )

    params["access_token"] = res.json().get("access_token")
    params["refresh_token"] = res.json().get("refresh_token")

    with open(f"{root_dir}/config.json", "w") as f:
        json.dump(params, f, indent=4)

2025-09-19 11:44:55,237 [INFO] __main__: Configuration loaded from /home/srmarshall/code/meathead-mode/config.json
2025-09-19 11:44:55,239 [DEBUG] urllib3.connectionpool: Starting new HTTPS connection (1): api.prod.whoop.com:443
2025-09-19 11:44:55,555 [DEBUG] urllib3.connectionpool: https://api.prod.whoop.com:443 "POST /oauth/oauth2/token HTTP/1.1" 200 None


In [2]:
from utils.resources.whoop import Whoop

Whoop.check_config(config_filepath=f"{root_dir}/config.json")

2025-09-19 11:11:07,700 [DEBUG] utils.resources.whoop: Configuration loaded from /home/srmarshall/code/meathead-mode/config.json
2025-09-19 11:11:07,700 [DEBUG] utils.resources.whoop: Loaded config: {'client_id': 'cb5f51d2-464d-409b-a5a3-35bd5e6718e6', 'client_secret': '949903e33e331e261e9880613122cad90d993641bd0702d974053d7adf79d24f', 'redirect_uri': 'http://localhost:8080/callback', 'access_token': 'CEeQKPj_Ya5XL6P4ytrsqyWh1SxCTpdokXoL8EDzjYg.ISy8XAkfdqBjFsrqGQOXvik-LdRDHdI7fBYmPNqa0OI', 'refresh_token': 'eCh0Gv0z3oQ3e_VvErf4M1SVon6LVCwMOmEcbkdFleU.NCmP9ikbk1tHiFeAqPqsfiL0IOLRJ1t06KcDHFefY-Y'}
2025-09-19 11:11:07,701 [INFO] utils.resources.whoop: All required configuration keys are present.


In [36]:
refresh_token(config_filepath=f"{root_dir}/config.json")

2025-09-19 11:42:22,996 [INFO] __main__: Configuration loaded from /home/srmarshall/code/meathead-mode/config.json
2025-09-19 11:42:22,997 [DEBUG] urllib3.connectionpool: Starting new HTTPS connection (1): api.prod.whoop.com:443
2025-09-19 11:42:23,447 [DEBUG] urllib3.connectionpool: https://api.prod.whoop.com:443 "POST /oauth/oauth2/token HTTP/1.1" 400 429
2025-09-19 11:42:23,448 [INFO] __main__: {'client_id': 'cb5f51d2-464d-409b-a5a3-35bd5e6718e6', 'client_secret': '949903e33e331e261e9880613122cad90d993641bd0702d974053d7adf79d24f', 'redirect_uri': 'http://localhost:8080/callback', 'access_token': None, 'refresh_token': None}


## Metrics

In [ ]:
import requests 

def get_records_by_date(records: list[dict], date: str = None) -> list:
    """
    Get all workouts from a specific day in YYYY-MM-DD format. If no date is provided, the default 
    is the most recent day with a workout.
    """
    if not date:
        date = records[0]['start'].split("T")[0] 

    records_by_date = 

    ##TODO: handle case where date is not in data -- fetch more records?

    return records_by_date

In [7]:
date = "2025-09-18"
context = {}

for record_type in ["workout", "sleep", "profile"]:
    records = Whoop.get_records(record_type, f"{root_dir}/config.json")

    if record_type == "profile":
        context[record_type] = records
        continue

    context[record_type] = get_records_by_date(records, date)

In [8]:
context

{'workout': [{'id': '8141d2ad-21ce-4345-bd7d-b7841f9b5011',
   'v1_id': 2057186215,
   'user_id': 28846686,
   'created_at': '2025-09-18T18:50:45.796Z',
   'updated_at': '2025-09-18T18:51:33.514Z',
   'start': '2025-09-18T18:04:30.481Z',
   'end': '2025-09-18T18:48:59.054Z',
   'timezone_offset': '-04:00',
   'sport_name': 'cycling',
   'score_state': 'SCORED',
   'score': {'strain': 8.659389,
    'average_heart_rate': 124,
    'max_heart_rate': 151,
    'kilojoule': 962.17175,
    'percent_recorded': 1.0,
    'distance_meter': None,
    'altitude_gain_meter': None,
    'altitude_change_meter': None,
    'zone_durations': {'zone_zero_milli': 188415,
     'zone_one_milli': 1933226,
     'zone_two_milli': 530629,
     'zone_three_milli': 16342,
     'zone_four_milli': 0,
     'zone_five_milli': 0}},
   'sport_id': 1},
  {'id': 'ecf496e9-2856-4d12-aaab-314f80df4258',
   'v1_id': 2056223864,
   'user_id': 28846686,
   'created_at': '2025-09-18T11:02:57.082Z',
   'updated_at': '2025-09-18T1